In [1]:
# 编码与哑变量

In [15]:
from sklearn.preprocessing import LabelEncoder # 对标签进行编
from sklearn.impute import SimpleImputer 
import pandas as pd
import numpy as np

In [16]:
data = pd.read_csv(r'/Users/liz/Code/jupyter-notebook/sklearn/2- Feature Engineering/Narrativedata.csv',index_col=0)
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         714 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [17]:
# 处理年龄的缺失值
Age = data.loc[:,'Age'].values.reshape(-1,1)
imp_age_mean = SimpleImputer()
Age = imp_age_mean.fit_transform(Age)

In [18]:
data.loc[:,'Age'] = Age
data.info()
data['Age']=data['Age'].astype('int')
data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         891 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


,Age,Sex,Embarked,Survived
0,22,male,S,No
1,38,female,C,Yes
2,26,female,S,Yes
3,35,female,S,Yes
4,35,male,S,No
...,...,...,...,...
886,27,male,S,No
887,19,female,S,Yes
888,29,female,S,No
889,26,male,C,Unknown


In [19]:
# 处理舱门的缺失——删除
data.dropna(axis=0,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
Age         889 non-null int64
Sex         889 non-null object
Embarked    889 non-null object
Survived    889 non-null object
dtypes: int64(1), object(3)
memory usage: 34.7+ KB


In [20]:
y = data.iloc[:,-1] # 取出特征，最后一列，标签允许是一维
le = LabelEncoder()  # 实例化
le = le.fit(y)
label = le.transform(y)
data.iloc[:,-1] = label
data.head()
le.classes_ # 查看标签中类别数量

array(['No', 'Unknown', 'Yes'], dtype=object)

In [21]:
data['Survived'][:5]

0    0
1    2
2    2
3    2
4    0
Name: Survived, dtype: int64

In [22]:
from sklearn.preprocessing import OrdinalEncoder # 处理特征专用

In [23]:
data_ = data.copy()
OrdinalEncoder().fit(data_.iloc[:,1:-1]).categories_ # 查看特征的类别
data_.iloc[:,1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:,1:-1])
data_.head()

,Age,Sex,Embarked,Survived
0,22,1.0,2.0,0
1,38,0.0,0.0,2
2,26,0.0,2.0,2
3,35,0.0,2.0,2
4,35,1.0,2.0,0


In [24]:
# one-hot 编码 向量形式
from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:,1:-1]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
Age         889 non-null int64
Sex         889 non-null object
Embarked    889 non-null object
Survived    889 non-null int64
dtypes: int64(2), object(2)
memory usage: 34.7+ KB


In [39]:
enc = OneHotEncoder()
enc = enc.fit(X)
result = enc.transform(X)
result

<889x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1778 stored elements in Compressed Sparse Row format>

In [40]:
# result = np.array(result.toarray())
result = result.toarray()


In [41]:
result

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [192]:
enc.get_feature_names() # 用于查看特征默认的名称

array(['x0_female', 'x0_male', 'x1_C', 'x1_Q', 'x1_S'], dtype=object)

In [201]:
newdata = pd.concat([data,pd.DataFrame(result)],axis=1)
newdata.head()

,Age,Sex,Embarked,Survived,0,1,2,3,4
0,22.0,male,S,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,female,C,2.0,1.0,0.0,1.0,0.0,0.0
2,26.0,female,S,2.0,1.0,0.0,0.0,0.0,1.0
3,35.0,female,S,2.0,1.0,0.0,0.0,0.0,1.0
4,35.0,male,S,0.0,0.0,1.0,0.0,0.0,1.0


In [203]:
newdata.drop(['Sex','Embarked'],inplace=True,axis=1)

In [205]:
newdata.columns = ['Age','Survived','Female','Male','Embarked_C','Embarked_Q','Embarked_S']

In [209]:
newdata.head()

,Age,Survived,Female,Male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,2.0,1.0,0.0,1.0,0.0,0.0
2,26.0,2.0,1.0,0.0,0.0,0.0,1.0
3,35.0,2.0,1.0,0.0,0.0,0.0,1.0
4,35.0,0.0,0.0,1.0,0.0,0.0,1.0
